In [ ]:
%pip install openai
%pip install requests # To get error text from http server
import openai
import requests
openai.api_key="sk-***"  # Replace this with your API key

## Define an API to retrieve log details from external site. 
### Using a static file stored in git in this case. 

In [11]:
# Definition of the function to get an error text
def get_error_message(msg_id="123456"):

    # This is a sample error message static text. This needs to be replaced by a db call or log file read
    url = "https://raw.githubusercontent.com/sakamakik-outlook/openai-hackday/main/data/errorlog.txt"

    return requests.get(url).text


# Function to be passed to OpenAI API
functions = [
    {
        "name": "get_error_message",
        "description": "Get the error message for a given message id",
        "parameters": {
            "type": "object",
            "properties": {
                "msg_id": {
                    "type": "string"
                }
            }
        },
    }
]

## Standard OpenAI API call without any log info. 
### GPT cannot answer the question, saying "I don't have access to personal information or specific messages."

In [23]:
messages = [{
    "role": "user", 
    "content": """
        What happened to my message 123456?
    """}]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages
    )

print(response["choices"][0]["message"]["content"])

I'm sorry, as an AI language model, I don't have access to personal information or specific messages. Therefore, I cannot tell you what happened to a specific message like 123456.


## Adding a function to the parameters. 
### 1. GPT will check if the function call is required. If yes, it returns "function_call" in the response
### 2. If "function_call" is returned, we execute the function and get the result
### 3. Run a new API call with the queston + result of function call (Error log in this case)


In [24]:
import json

messages = [{
    "role": "user", 
    "content": """
        What happened to my message 123456?
    """}]

# Step 1: call the API
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages, 
        functions=functions,   # <---- this is the new part. Pass the functions to the API.
        function_call="auto",
        )

response_message = response["choices"][0]["message"]

# Step 2: check if GPT wanted to call a function
if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
                "get_error_message": get_error_message,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
                msg_id=function_args.get("msg_id")
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
                {
                "role": "function",
                "name": function_name,
                "content": function_response,
                }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-0613",
                messages=messages,
        )  # get a new response from GPT where it can see the function response
        print(second_response)

{
  "id": "chatcmpl-7dtjoYElkzLJkj07iPiRmgR9mtUSG",
  "object": "chat.completion",
  "created": 1689743404,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Based on the error message, it appears that there was a connection failure while processing your message with ID 123456. The error indicates that there was a problem with the database connection, specifically with the host name or IP address \"10.1.1.1\" and the service name or port number 446. The error code is -4498 and the SQL state is 08506. \n\nIt seems that the connection was re-established, but there may have been some impact on the processing of your message. You may need to review the logs or contact the relevant support team to investigate further and determine the status of your message."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1509,
    "completion_tokens": 128,
    "total_tokens": 1637
